<a href="https://colab.research.google.com/github/notnikhilreddy/Word-Vectorization/blob/master/Word_Vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install gensim

In [0]:
!wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles1.xml-p1p30303.bz2

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [23]:
# CONVERT RAW WIKIPEDIA DUMP FILE INTO CORPUS

import sys
from gensim.corpora import WikiCorpus
from IPython.display import HTML, display
from datetime import datetime

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))


in_f = "word2vec/enwiki-latest-pages-articles1.xml-p1p30303.bz2"
out_f = "word2vec/wiki_en2.txt"
print("Started at :", datetime.now())
articles_count = display(progress(0, 6074991), display_id=True)
# make_corpus(in_f, out_f)

"""Convert Wikipedia xml dump file to text corpus"""
wiki = WikiCorpus(in_f)

print("Started For loop")
i = 0
for text in wiki.get_texts():
	output = open(out_f, 'a')
	output.write(bytes(' '.join(text), 'utf-8').decode('utf-8') + '\n')
	articles_count.update(progress(i, 100))
	i = i + 1
	if (i % 10000 == 0):
		print("Articles Finished :", i)
	output.close()

print('Processing complete!')
print("Finished at :", datetime.now())

Started at : 2020-05-15 04:09:28.493649


Started For loop
Articles Finished : 10000
Processing complete!
Finished at : 2020-05-15 04:16:23.389175


In [5]:
# GENERATE UNIQUE WORDS SET FROM CORPUS

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

all_words_count = 0;
unique_words = set()

# all_words_file = open(out_f, 'w')
input_file = "/content/drive/My Drive/Projects/word2vec/data/wiki_en_15000.txt"

line_no = 0
with open (input_file, "r") as corpus:
  for line in corpus:
    line_no += 1
    tokenize_word = word_tokenize(line.strip())
    for word in tokenize_word:
      unique_words.add(word)
      all_words_count += 1
    if(line_no % 500 == 0):
      print("Articles: " + str(line_no) + "\t" + "All words: " + str(all_words_count) + "\t" + "Unique words: " + str(len(unique_words)))
      # all_words_file.write(bytes(word, 'utf-8').decode('utf-8') + '\n')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Articles: 500	All words: 1842184	Unique words: 84040
Articles: 1000	All words: 3352260	Unique words: 121275
Articles: 1500	All words: 4706629	Unique words: 148760
Articles: 2000	All words: 6421775	Unique words: 173107
Articles: 2500	All words: 8178528	Unique words: 196001
Articles: 3000	All words: 9851328	Unique words: 216955
Articles: 3500	All words: 11400816	Unique words: 235381
Articles: 4000	All words: 13059281	Unique words: 254668
Articles: 4500	All words: 14764654	Unique words: 272056
Articles: 5000	All words: 16220838	Unique words: 290112
Articles: 5500	All words: 17886506	Unique words: 308630
Articles: 6000	All words: 19539454	Unique words: 327016
Articles: 6500	All words: 21411232	Unique words: 343665
Articles: 7000	All words: 23345023	Unique words: 361409
Articles: 7500	All words: 25079423	Unique words: 377340
Articles: 8000	All words: 26804319	Unique words: 392966
A

In [0]:
# WRITE UNIQUE WORDS TO A FILE (not really useful)

unique_words = sorted(unique_words)
unique_words_file = "/content/drive/My Drive/Projects/word2vec/data/unique_words_file.txt"
with open(unique_words_file, "w") as f:
  for word in unique_words:
    f.write(bytes(word, 'utf-8').decode('utf-8') + '\n')

In [7]:
print("No. of words:\t" + str(all_words_count))
# unique_words = set(all_words)
print("No. of unique words:\t" + str(len(unique_words)))

# print(unique_words[:100])
# lines = 0
# with open(unique_words_file, "r") as uwords:
#   for line in uwords:
#     lines += 1
# print(lines)

No. of words:	49300120
No. of unique words:	571050


In [4]:
# FIT A TOKENIZER OR UNIQUE WORDS

from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower=True, split=' ', oov_token="0")
tokenizer.fit_on_texts([list(unique_words)])

Using TensorFlow backend.


NameError: ignored

In [0]:
# CREATE A TOKENIZED DATABASE

output = open("/content/drive/My Drive/Projects/word2vec/data/wiki_en_15000_tokenized.txt", "w")

with open("/content/drive/My Drive/Projects/word2vec/data/wiki_en_15000.txt", "r") as corpus:
  for line in corpus:
    # print(line)
    sequence = tokenizer.texts_to_sequences([line])
    # print+(sequence)
    # break
    for i in sequence[0]:
      output.write(bytes(str(i), 'utf-8').decode('utf-8') + ' ')
    output.write('\n')

output.close()

In [11]:
print(len(tokenizer.word_index))

571051


In [0]:
unique_words_count = 571051

In [12]:
import numpy as np
import tensorflow as tf
import csv

def to_one_hot(data_point_index, vocab_size):
  temp = np.zeros(vocab_size)
  if data_point_index >= 0:
    temp[data_point_index] = 1
  return temp

csvfile = open('/content/drive/My Drive/Projects/word2vec/data/wiki_en_15000_onehot.csv', 'w', newline='')
csvwriter = csv.writer(csvfile, delimiter=',', lineterminator='\n')
# csvfile_y = open('/content/drive/My Drive/Projects/word2vec/data/wiki_en_15000_y_train.csv', 'w', newline='')
# csvwriter_y = csv.writer(csvfile_y, delimiter=',', lineterminator='\n')

with open("/content/drive/My Drive/Projects/word2vec/data/wiki_en_15000_tokenized.txt", 'r') as data:
  for line in data:
    # print(line.strip().split(' '))
    # break
    words = [int(i) for i in line.strip().split(' ')]
    words_count = len(words)

    for word in words:
      onehot = to_one_hot(word, unique_words_count)
      csvwriter.writerow(onehot)

csvfile.close()
# csvfile_y.close()
    # unique_words_count = len()

    # x_train = []
    # y_train = []

    # for window in range(words_count):
    #   one = to_one_hot(words[window], unique_words_count)
    #   two = to_one_hot(words[window+1], unique_words_count)
    #   three = to_one_hot(-1, unique_words_count)
    #   four = to_one_hot(words[window+3], unique_words_count)
    #   five = to_one_hot(words[window+4], unique_words_count)

    #   # group = one.add(two).append(three).append(four).append(five)
    #   x_train = np.concatenate((one, two, three, four, five), axis=0)
    #   y_train = to_one_hot(words[window+2], unique_words_count)

    #   csvwriter_x.writerow(x_train)
    #   csvwriter_y.writerow(y_train)
      # x_train.append(group)
      # y_train.append(to_one_hot(words[window+2], unique_words_count))
      # break

    # x_train = np.asarray(x_train)
    # y_train = np.asarray(y_train)
    # break



KeyboardInterrupt: ignored